In [1]:
print('''

Ask a home buyer to describe their dream house, 
and they probably won't begin with the height
of the basement ceiling or the proximity to
an east-west railroad. But this playground competition's dataset 
proves that much more influences price negotiations than 
the number of bedrooms or a white-picket fence.

With 79 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this competition challenges you to predict the final price of each home.

It is your job to predict the sales price for each house. 
For each Id in the test set, you must predict the value of the SalePrice variable. 

Metric: 
Submissions are evaluated on Root-Mean-Squared-Error (RMSE)
between the logarithm of the predicted value and the logarithm
of the observed sales price. 

(Taking logs means that errors in predicting expensive houses 
and cheap houses will affect the result equally.)

''')



Ask a home buyer to describe their dream house, 
and they probably won't begin with the height
of the basement ceiling or the proximity to
an east-west railroad. But this playground competition's dataset 
proves that much more influences price negotiations than 
the number of bedrooms or a white-picket fence.

With 79 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this competition challenges you to predict the final price of each home.

It is your job to predict the sales price for each house. 
For each Id in the test set, you must predict the value of the SalePrice variable. 

Metric: 
Submissions are evaluated on Root-Mean-Squared-Error (RMSE)
between the logarithm of the predicted value and the logarithm
of the observed sales price. 

(Taking logs means that errors in predicting expensive houses 
and cheap houses will affect the result equally.)




In [2]:
import pandas as pd
# import the training and holdout data
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
data = train.append(test)

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [3]:
import numpy as np
logtransform = ['SalePrice']
for i in logtransform: 
    data[i] = np.log(np.array(data[i]))

In [4]:
# CATBOOST FEATURE PREP

data = data[data['SalePrice'].notna()]
data.fillna(-999, inplace = True)
X = data.drop(['SalePrice', 'Id'], axis =1)
y = data['SalePrice']
cat_features = np.where(X.dtypes == 'object')[0]

# TRAIN TEST SPLIT

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state=42)
from catboost import Pool, CatBoostRegressor, cv

train_pool = Pool(X_train, y_train, cat_features = cat_features)
test_pool = Pool(X_test, y_test, cat_features = cat_features)

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [5]:
# TRAIN TEST SPLIT

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, random_state=42)

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [6]:
from catboost import Pool, CatBoostRegressor
# more on MAE - https://heartbeat.fritz.ai/5-regression-loss-functions-all-machine-learners-should-know-4fb140e9d4b0
cat_model = CatBoostRegressor(
    loss_function='MAE',
    random_seed = 15,
    eval_metric = 'RMSE',
    verbose = 300, 
    max_ctr_complexity=15, 
    iterations = 1500
     )

cat_model.fit(
    train_pool,
    eval_set = test_pool, 
    use_best_model = True
    )

0:	learn: 12.0224990	test: 11.9908746	best: 11.9908746 (0)	total: 126ms	remaining: 3m 9s
300:	learn: 7.5951035	test: 7.5549990	best: 7.5549990 (300)	total: 6.94s	remaining: 27.6s
600:	learn: 3.1237248	test: 3.0858741	best: 3.0858741 (600)	total: 8.11s	remaining: 12.1s
900:	learn: 0.1279841	test: 0.1538737	best: 0.1538737 (900)	total: 14.2s	remaining: 9.46s
1200:	learn: 0.0842793	test: 0.1403735	best: 0.1403211 (1199)	total: 27.3s	remaining: 6.8s
1499:	learn: 0.0605608	test: 0.1370624	best: 0.1365999 (1469)	total: 41.1s	remaining: 0us

bestTest = 0.1365999165
bestIteration = 1469

Shrink model to first 1470 iterations.


In [7]:
print('''
RMSE has been minimised to 0.1345 and R-Square of Test and Train datasets is above 0.90. 
what worked - > 
1. using Catboost to allow handling of multiple categorical features
2. allowing Catboost to explore more permutations of categorical features
3. using MAE instead of RMSE/R2 in Loss function. MAE helps model ignore influence of outliers. 
All in all, House Prices are easy to predict if we are given such rich data.
''')


RMSE has been minimised to 0.1345 and R-Square of Test and Train datasets is above 0.90. 
what worked - > 
1. using Catboost to allow handling of multiple categorical features
2. allowing Catboost to explore more permutations of categorical features
3. using MAE instead of RMSE/R2 in Loss function. MAE helps model ignore influence of outliers. 
All in all, House Prices are easy to predict if we are given such rich data.

